In [48]:
import pandas as pd
from difflib import SequenceMatcher
import numpy as np
import re

In [50]:
cvspath = 'data/PnG_Pampers_NL_Product_Feed_OG.csv'
tamppath = 'data/PnG_Pampers_NL_Product_Feed_OG.xlsx'

ccPath = 'data/country_codes.xlsx'
lcPath = 'data/lang_codes.xlsx'

sellerFeedPath = 'data/seller-feeds/ApexToolGroup_Store_Feed.xlsx'
productFeedPath = 'data/product-feeds/PnG_Pampers_AE_Product_Feed.xlsx'

# data = pd.read_excel(tamppath, converters={'SKU': lambda x: str(x)})
sfData = pd.read_excel(sellerFeedPath)
pfData = pd.read_excel(productFeedPath)

countryCodes = pd.read_excel(ccPath)
langCodes = pd.read_excel(lcPath)

sfData

,Country,sellerName,sellerID,storeName,storeID,productGroup,sku,address,address2,city,...,Wednesday,Thursday,Friday,Saturday,Sunday,latitude,longitude,sellerUrl,sellerUrlLabel,miscText
0,CA,Acklands Grainger,100000,Acklands Grainger Inc - AC0,AC0,US GearWrench | US Jobox,NaN,3 Kingsview Rd,NaN,Airdrie,...,8:00am - 5:00pm,8:00am - 5:00pm,8:00am - 5:00pm,NaN,NaN,51.083163,-114.012,NaN,NaN,NaN
1,CA,Acklands Grainger,100000,Acklands Grainger Inc - AA9,AA9,US GearWrench | US Jobox,NaN,5802 - 50 Ave,NaN,Bonnyville,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,55.278487,-114.773,NaN,NaN,NaN
2,CA,Acklands Grainger,100000,Acklands Grainger Inc - AC4,AC4,US GearWrench | US Jobox,NaN,3401 - 19 St NE Bay #10,NaN,Calgary,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,42.931627,-81.2125,NaN,NaN,NaN
3,CA,Acklands Grainger,100000,Acklands Grainger Inc - AC1,AC1,US GearWrench | US Jobox,NaN,4340 Manhattan Rd SE,NaN,Calgary,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,51.014805,-114.052,NaN,NaN,NaN
4,CA,Acklands Grainger,100000,Acklands Grainger Inc - AC2,AC2,US GearWrench | US Jobox,NaN,4949 - 61 Ave SE,NaN,Calgary,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,50.998624,-113.963,NaN,NaN,NaN
5,CA,Acklands Grainger,100000,Acklands Grainger Inc - ABA,ABA,US GearWrench | US Jobox,NaN,3814 - 47 Ave,NaN,Camrose,...,8:00am - 5:00pm,8:00am - 5:00pm,8:00am - 5:00pm,NaN,NaN,52.323749,-112.691,NaN,NaN,NaN
6,CA,Acklands Grainger,100000,Acklands Grainger Inc - AE3,AE3,US GearWrench | US Jobox,NaN,15986 - 118 Ave,NaN,Edmonton,...,7:30am - 5:00pm,7:30am - 5:00pm,7:30am - 5:00pm,NaN,NaN,53.565301,-113.321,NaN,NaN,NaN
7,CA,Acklands Grainger,100000,Acklands Grainger Inc - AE6,AE6,US GearWrench | US Jobox,NaN,8468 Roper Rd,NaN,Edmonton,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,53.489776,-113.45,NaN,NaN,NaN
8,CA,Acklands Grainger,100000,Acklands Grainger Inc - AA3,AA3,US GearWrench | US Jobox,NaN,4421 - 2nd Ave,NaN,Edson,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,53.584814,-116.417,NaN,NaN,NaN
9,CA,Acklands Grainger,100000,Acklands Grainger Inc - AP1,AP1,US GearWrench | US Jobox,NaN,250 Mackay Cres,NaN,Fort Mcmurray,...,7:00am - 5:00pm,7:00am - 5:00pm,7:00am - 5:00pm,NaN,NaN,56.669436,-111.332,NaN,NaN,NaN


In [6]:
# removes whitespace
def remove_whitespace(df):
    dataDict = df.to_dict('records')
    for x in dataDict:
        for value in x:
            try:
                x[value] = " ".join(x[value].split())

            except:
#               passes on ints
                pass
    data = pd.DataFrame.from_dict(dataDict)
    return data

In [194]:
# verifies upc is numeric and is 13-15 digits lon. Takes in df and upc column name.
def upc_validation(df,upcColumnName):
    badUPCs = []
    dataDict = df.to_dict('records')
    for x in dataDict:
        upcLen = len(str(x[upcColumnName]))
#       append non numerical upc's to badUPCs
        try:
            x[upcColumnName] = int(float(x[upcColumnName]))
        except:
            badUPCs.append(x)
#       verify length of upc
        if upcLen < 13 or upcLen > 15:
            if x not in badUPCs:
                badUPCs.append(x)
            else:
                pass
        else:
            pass
        
#   remove upc's with nan or empty values. Since value is alound to be empty
    for item in badUPCs[:]:
        strUpc = str(item[upcColumnName])
        if strUpc == "nan" or strUpc == "":
            badUPCs.remove(item)
        else:
            pass
        
    for x in badUPCs:
        if x in dataDict[:]:
            dataDict.remove(x)
#                 print(x)
        else:
            pass
        
        dfGood = pd.DataFrame(dataDict)
        dfBad =  pd.DataFrame(badUPCs)  
        
    return dfGood, dfBad

df1, dfBadUPC = upc_validation(data,"upc")
# print(dfBadUPC.to_string(),df1.to_string())
dfBadUPC

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
0,NL,Pampers,4015400833826,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,Pampers,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 4 9/14 kg,86 luiers (voor ongeveer 20 dagen),ES,NaN,NaN
1,NL,Pampers,4015400833857,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4000000010,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 4 9/14 kg,96 luiers (voor ongeveer 22 dagen),ES,NaN,NaN
2,NL,Pampers,4015400834168,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,40000000011,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 4+ 10/15 kg,112 luiers (voor ongeveer 25 dagen),ES,NaN,NaN
3,NL,Pampers,4015400566748,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,400000000012,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 4+ 10/15 kg,152 luiers (voor ongeveer 34 dagen),ES,NaN,NaN
4,NL,Pampers,4015400833918,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4000000000000160,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 4+ 10/15 kg,24 luiers (voor ongeveer 6 dagen),ES,NaN,NaN


In [55]:
# takes in a list of good brands and a list of questionable brands and spits out a Dict of brands to replace
def comparison_swap(good, questionable):
    
    gthreshold = []
    qthreshold = [] 
    ratioDict = []

    
#   Creates two lists of the same len that are aligned via index based on character similarity.
    for g in list(good):
        for q in questionable:
            ratiothresh = SequenceMatcher(None, g, q).ratio()
            if ratiothresh > 0.50:
                gthreshold.append(g)
                qthreshold.append(q)
            else:
                pass
#     print(gthreshold,qthreshold,"\n")

#   Send to lowercasee for case insensitive comparison
    gL = (word.lower() for word in gthreshold)
    qL = (word.lower() for word in qthreshold)
    
#   loop grab ratio of each pair and change if necessary
    for index, (wordg, wordq) in enumerate(zip(gL,qL)):
#       check if values are the same before attempting the switch
        if qthreshold[index] is gthreshold[index]:
            same = True
        else:
            same = False
        
        ratio = SequenceMatcher(None, wordg, wordq).ratio()
        
#       switch the questionable brand to the good brand
        if ratio > 0.80 and same == False:
            replaced = qthreshold[index]
            qthreshold[index] = gthreshold[index]
        else:
            replaced = "N/A"
            
            
        ratioDict.append({
            "ratio": ratio, 
            "examined": gthreshold[index],
            "replaced": replaced,
            "with": qthreshold[index]})
    
    return ratioDict



In [103]:
# finds brands that have a low count and attempts to update them based on similar barnd switch. 
def outlier_detect(df,brandColumnName):

    dataDict = df.to_dict('records')
    q_brands = []
    g_brands = []
    brandDict = []
    
#   count frequency of brands
    brandCounts = df[brandColumnName].value_counts()
    
#   determine cut off as a perct of mean
    mean = np.mean(brandCounts)
    cut_off = 0.10 * mean
    

#   append brand names to q_brand that appear less than the cutoff.
    for key, value in brandCounts.items():
        if value < cut_off:
            q_brands.append(key)
        else:
            g_brands.append(key)
            
#   See similar_brand_switch       
    flagged_brands = comparison_swap(g_brands, q_brands)
    
#   create list of rows that contain the question brand names.
    for x in dataDict:
        if x[brandColumnName] in q_brands:
            brandDict.append(x)
            
#   Replace brands in dataDict that have been id'd in flagged_brands for replacement
    for y in brandDict:
        for z in flagged_brands:
            if y[brandColumnName] == z["replaced"]:         
                y[brandColumnName] = z["with"]
                
    
    branddf = pd.DataFrame(dataDict) 
    
#   get value counts again. Send any stragglers to error df
    brandCounts2 = branddf[brandColumnName].value_counts()
    
    lowCountIndex = []
    
    for key, value in brandCounts2.items():
        if value < 2:
            indx = branddf.index[branddf[brandColumnName] == key].tolist()
            lowCountIndex.append(indx)  

#   flatten lowCountIndex that comes back as a 1x2 list
    flatList = []
    
    for sublist in lowCountIndex:
        for indx in sublist:
            flatList.append(indx)
            
    uncorrected = branddf.iloc[flatList]
    
    branddf = branddf.drop(flatList)
    
    return branddf, uncorrected
        
good, bad = outlier_detect(data, "manufacturerName")
bad

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
185,NL,Salmmers,8001090834966,NaN,NL Pampers Pure Protection,Pampers Pure Protection,8001090834966,NaN,https://www.pampers.nl/luiers-babydoekjes-pure...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-pure-protection-luiers,5,24 luiers (voor +/- 6 dagen),en,NaN,NaN


In [47]:
# finds duplicate skus,altskus, or upc against country or lang. Accepts dataframe and column name as string
def dup_validation(df,dupColumnaName,againstColumnName,*columns):
    dataDict = df.to_dict('records')
    
    colNameList = [dupColumnaName, againstColumnName]
    
#   adds optional args to colNameList
    for col in columns:
        colNameList.append(col)
    
    print(colNameList)
    
#   duplicated returns second instance of value as pd.series
    dup = df.duplicated(colNameList)
    
#   list of indexes to use in loc
    indexList = []
    
#   iterate through boolean duplicate data to return indexs of dups
    for key, value in dup.items():
        if value == True:
            indexList.append(key)
            
    dfClean = df.drop(df.index[indexList])
    
#   df of dups pulled by in dup
    dupDF = df.iloc[indexList]
    
    
######################################################################################################################
# logic to remove all instaces of dups. 

# #   Send df sku column to list
#     sku_list = rows[dupColumnaName].tolist()
    
#     dupList = []
    
# #   find the dup and the original from df and append to list
#     for x in dataDict:
#         if x[dupColumnaName] in sku_list:
#             dupList.append(x)


#     dfBad =  pd.DataFrame(dupList) 
######################################################################################################################

            
    
    dfGood = dfClean
    dfBad = dupDF

                
    return dfGood, dfBad
    
df2, dfDup = dup_validation(sfData, "sellerID", "sellerName", "storeName", "storeID")   

dfDup

['sellerID', 'sellerName', 'storeName', 'storeID']


,Country,sellerName,sellerID,storeName,storeID,productGroup,address,address2,city,state,zip,phone,Website
1,US,2Guyz Boutique for Dogs and Cats,1926,NaN,NaN,All Products,1453 South A St.,Suites 1000 A-D,Elwood,IN,46036,(765) 356-7387,http://elwoodpetshop.com


In [241]:
# wont work on two word categorys
# pipes altskus and categorys. Takes in df and column name you want to pipe. 
def pipe_it(df,columnNametoPipe):
    
    dataDict = df.to_dict('records')
    
#   splits on non word characters and joins with |
    for x in dataDict:
        pipeStr = str(x[columnNametoPipe])
        regexList = re.split(r"\W+(?<!-)(?<!\s)\s*", pipeStr)
        x[columnNametoPipe] = "|".join(regexList)
        
    asDf = pd.DataFrame(dataDict)

    return asDf

pipeDF = pipe_it(data,"altSku")
pipeDF

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
0,nl,Pamper,4015400833079,1|420blazit|wow,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833079,2019-07-20 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,33 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
1,NL,Pamperz,8001090268617,1asdf|asdf|pipe,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090088185,2019-07-21 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,42 luiers (voor ongeveer 8 dagen),ES,NaN,NaN
2,GE,pampers,4015400833154,this is made of altskus,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336354,2019-07-22 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,46 luiers (voor ongeveer 9 dagen),ES,NaN,NaN
3,US,Pampers,4015400833154,4015400833154|hat,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833154,2019-07-23 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,58 luiers (voor ongeveer 11 dagen),ES,NaN,NaN
4,NL,Pampers,8001090268617,1234|232|234,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-12 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,100 luiers (voor ongeveer 20 dagen),ES,NaN,NaN
5,NL,Pampers,4015400833420,a|D|B,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-25 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,112 luiers (voor ongeveer 23 dagen),ES,NaN,NaN
6,NL,Pampers,4015400833444,1|2|3,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833444,2019-07-26 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,136 luiers (voor ongeveer 28 dagen),ES,NaN,NaN
7,NL,Pampers,4015400566663,asdf,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400566663,2019-07-27 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,198 luiers (voor ongeveer 40 dagen),ES,NaN,NaN
8,NL,Pampers,4015400833260,hi wut-up,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833260,2019-07-28 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,30 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
9,NL,Pampers,4015400833307,nan,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833307,07/32/2019,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,34 luiers (voor ongeveer 7 dagen),ES,NaN,NaN


In [18]:
# verify country code are accurate and formated correctly. Returns good and bad df
def country_code_validation(df,ccdf):
    
    dataDict = df.to_dict('records')
    badCountries = []
    
#   send the df columns to lists
    cCodes = ccdf["Code"].tolist()
    cNames = ccdf["Name"].tolist()
    
    
    for index, x in enumerate(dataDict):
        
        country = x["Country"]
        
        CapitalTest = country.isupper()
        
#       Capitalizes names in order to make a match against cNames
        if CapitalTest == False and len(country) > 2:
            country = country.capitalize()
            
#       makes sure country codes are uppercased
        if len(country) == 2:
            x["Country"] = country.upper()
            
            # reassingn country after the switch for later use       
            country = x["Country"]
            
#       change country name to country code if name in CNames   
        if country in cNames:
            nameIndex = cNames.index(country)
            x["Country"] = cCodes[nameIndex]
            
#       Send x to badCountries if not in cCodes and cNames
        elif country not in cCodes:
            badCountries.append(x)
            dataDict.remove(x)
            
    goodDf = pd.DataFrame(dataDict) 
    badDf = pd.DataFrame(badCountries)
   
    return badDf, goodDf

bad, good = country_code_validation(data1, countryCodes)
good

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage
0,AE,Pampers,4015400406907,NaN,AE DIAPERS Pampers new baby-dry,DIAPERS Pampers new baby-dry,4015400406907,NaN,https://www.pampers.ae/products/new-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,new-baby-dry,Size 1 (2 to 5 kg),21,EN
1,AE,Pampers,4015400406969,NaN,AE DIAPERS Pampers new baby-dry,DIAPERS Pampers new baby-dry,4015400406969,NaN,https://www.pampers.ae/products/new-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,new-baby-dry,Size 1 (2 to 5 kg),66,EN
2,AE,Pampers,4015400405788,NaN,AE DIAPERS Pampers new baby-dry,DIAPERS Pampers new baby-dry,4015400405788,NaN,https://www.pampers.ae/products/new-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,new-baby-dry,Size 2 (3 to 8 kg),23,EN
3,AE,Pampers,4015400405887,NaN,AE DIAPERS Pampers new baby-dry,DIAPERS Pampers new baby-dry,4015400405887,NaN,https://www.pampers.ae/products/new-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,new-baby-dry,Size 2 (3 to 8 kg),64,EN
4,AE,Pampers,4015400406150,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,4015400406150,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 3 (6 to10 kg),46,EN
5,AE,Pampers,4015400406211,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,4015400406211,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 3 (6 to10 kg),68,EN
6,AE,Pampers,4015400406181,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,4015400406181,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 3 (6 to10 kg),88,EN
7,AE,Pampers,6291105653503,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,6291105653503,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 3 (6 to10 kg),272,EN
8,AE,Pampers,4015400751403,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,4015400751403,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 4 (9 to 14 kg),16,EN
9,AE,Pampers,4015400406365,NaN,AE DIAPERS Pampers Baby-Dry,DIAPERS Pampers Baby-Dry,4015400406365,NaN,https://www.pampers.ae/products/pampers-baby-dry,https://azcdn.multibrand.pgsitecore.com/en-AE/...,AE Pampers,Nappies,pampers-baby-dry,Size 4 (9 to 14 kg),44,EN


In [28]:
# Takes in df and column name. Returns good df with bad dates removed and bad df with errors
def validate_date(df, DateColumnName):
    
    
    dataDict = df.to_dict('records')
    badDate = []
    
#   try to covert x to date. If not send to badDate list
    for x in dataDict:
        try:
            date = x[DateColumnName].date()
            x[DateColumnName] = date
        except:
            badDate.append(x)

#   Remove bad dates from dataDict
    for x in dataDict:
        if x in badDate:
            dataDict.remove(x)
    
    gooddf = pd.DataFrame(dataDict)
    baddf = pd.DataFrame(badDate)
         
    return gooddf, baddf

good, bad = validate_date(data, "releaseDate")
good

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
0,nl,Pamper,4015400833079,1 | 420blazit|wow,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833079,2019-07-20,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,33 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
1,NL,Pamperz,8001090268617,"1asdf, asdf|pipe",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090088185,2019-07-21,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,42 luiers (voor ongeveer 8 dagen),ES,NaN,NaN
2,GE,pampers,4015400833154,this is made of altskus,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336354,2019-07-22,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,46 luiers (voor ongeveer 9 dagen),ES,NaN,NaN
3,US,Pampers,4015400833154,"4015400833154, hat",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833154,2019-07-23,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,58 luiers (voor ongeveer 11 dagen),ES,NaN,NaN
4,NL,Pampers,8001090268617,"1234, 232, 234",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-12,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,100 luiers (voor ongeveer 20 dagen),ES,NaN,NaN
5,NL,Pampers,4015400833420,a|D|B,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-25,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,112 luiers (voor ongeveer 23 dagen),ES,NaN,NaN
6,NL,Pampers,4015400833444,1|2|3,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833444,2019-07-26,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,136 luiers (voor ongeveer 28 dagen),ES,NaN,NaN
7,NL,Pampers,4015400566663,asdf,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400566663,2019-07-27,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,198 luiers (voor ongeveer 40 dagen),ES,NaN,NaN
8,NL,Pampers,4015400833260,hi wut-up,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833260,2019-07-28,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,30 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
9,NL,Pampers,8001090336408,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336408,2019-07-08,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,38 luiers (voor ongeveer 8 dagen),ES,NaN,NaN


In [299]:
# verify country code are accurate and formated correctly. Returns good and bad df
def lang_code_validation(df,ccdf):
    
    dataDict = df.to_dict('records')
    badCountries = []
    
    lCodes = ccdf["639-1"].tolist()
    lNames = ccdf["ISO language name"].tolist()
    
    lCodes = [x.upper() for x in lCodes]
    
    for index, x in enumerate(dataDict):
        
        lang = x["resourceLanguage"]
        
        CapitalTest = lang.isupper()
        
#       Capitalizes names in order to make a match against cNames
        if CapitalTest == False and len(lang) > 2:
            lang = lang.capitalize()
            
#       makes sure country codes are uppercased
        if len(lang) == 2:
            x["resourceLanguage"] = lang.upper()
            
            # reassingn country after the switch for later use       
            lang = x["resourceLanguage"]
            
#       change country name to country code if name in CNames   
        if lang in lNames:
            nameIndex = lNames.index(lang)
            x["resourceLanguage"] = lCodes[nameIndex]
            
#       Send x to badCountries if not in cCodes and cNames
        elif lang not in lCodes:
            badCountries.append(x)
            dataDict.remove(x)
            
    goodDf = pd.DataFrame(dataDict) 
    badDf = pd.DataFrame(badCountries)
   
    return goodDf, badDf

good, bad = lang_code_validation(data1, langCodes)
good

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,Size,resourceLanguage,resourceKey
0,US,Downy,37000987045,NaN,US Downy,Downy Infusions Amber Blossom In-Wash Scent Bo...,NaN,NaN,https://downy.com/en-US/fabric-softener/infusi...,https://cdn.pioneer.p,US Downy,Extra Scent,Scent Beads,Extra Scent,9.7 oz,EN,NaN
1,US,Downy,37000987052,NaN,US Downy,Downy Infusions Amber Blossom In-Wash Scent Bo...,NaN,NaN,https://downy.com/en-US/fabric-softener/infusi...,https://cdn.pioneer.pgsitecore.commage Title_T...,US Downy,Extra Scent,Scent Beads,Extra Scent,13.2 oz,EN,NaN
2,US,Downy,37000987069,NaN,US Downy,Downy Infusions Amber Blossom In-Wash Scent Bo...,037000987069343434343,NaN,https://downy.com/en-US/fabric-softener/infusi...,https://www.amazon.com,US Downy,Extra Scent,Scent Beads,Extra Scent,19.5 oz,EN,NaN
3,US,Downy,37000904427,NaN,US Downy,Downy Infusions Amber Blossom In-Wash Scent Bo...,NaN,NaN,https://downy.com/en-US/fabric-softener/infusi...,https://www.amazon.com,US Downy,Extra Scent,Scent Beads,Extra Scent,36.26 oz,EN,NaN
4,uS,Downy,37000635222,NaN,US Downy,Downy Protect Refresh Active Fresh Liquid Fabr...,NaN,NaN,https://downy.com/en-US/fabric-softener/protec...,https://www.amazon.com,US Downy,Odor Prevention,Liquid Fabric Conditioner,Odor Prevention,"44 oz, 52 loads",EN,NaN
5,US,Downy,37000636373,NaN,US Downy,Downy Protect Refresh Active Fresh Liquid Fabr...,NaN,NaN,https://downy.com/en-US/fabric-softener/protec...,https://www.amazon.com,US Downy,odor Prevention,Liquid Fabric Conditioner,Odor Prevention,"67 oz, 78 loads",EN,NaN
6,US,Downy,37000636373,NaN,US Downy,Downy Protect Refresh Active Fresh Liquid Fabr...,NaN,NaN,https://downy.com/en-US/fabric-softener/protec...,https://www.amazon.com,US Downy,Odor Prevention,Liquid Fabric Conditioner,Odor Prevention,"77 oz, 90 loads",EN,NaN
7,uS,Downy,37000636991,NaN,US Downy,Downy Protect Refresh Active Fresh Liquid Fabr...,NaN,NaN,https://downy.com/en-US/fabric-softener/protec...,https://www.amazon.com,US Downy,Odor Prevention,Liquid Fabric Conditioner,Odor Prevention,"90 oz, 105 loads",EN,NaN
8,US,Downy,37000637509,NaN,US Downy,Downy Protect Refresh Active Fresh Liquid Fabr...,NaN,NaN,https://downy.com/en-US/fabric-softener/protec...,https://www.amazon.com,US Downy,Odor Prevention,Liquid Fabric Conditioner,Odor Prevention,"103 oz, 120 loads",EN,NaN
9,US,Downy,37000357162,NaN,US Downy,Downy Ball,NaN,NaN,https://downy.com/en-US/fabric-softener/DownyBall,https://www.amazon.com,US Downy,Softness,Downy Ball,Softness,Downy Ball,EN,NaN


In [16]:
# validate product group
def productGroup_validation(df, columnName, brandColumnName):
    
    dataDict = df.to_dict('records')
    
    badPG = []
    
    for x in dataDict:
        pg = x[columnName]
        country = x["Country"]
        brand = x[brandColumnName]
        countryBrand = f"{country} {brand}"
       
        if pg != countryBrand:
            badPG.append(x)
            
    for y in badPG:
        if y in dataDict:
            dataDict.remove(y)
    
    return dataDict, badPG

df2, pgdf = productGroup_validation(data1, "productGroup", "manufacturerName")
pgdf

[]

In [5]:
def leading_zeros(df,skuColumnName):
    
#   declared here so that when the function is not triggered the original df is returned
    lzDf = df
    
    dataDict = df.to_dict('records')
    
#   grab total number of rows
    skuCount = len(dataDict)
    
#   set threshold to trigger function
    triggerThreshold = skuCount * 0.60
    
    counter = 0

#   count num of skus with leading zeros
    for x in dataDict:
        
        sku = x[skuColumnName]
        
        skuLen = len(sku) - len(sku.lstrip('0'))
        
        if skuLen != 0:
            counter +=1
            
#   trigger function if counter exceeds thresh       
    if counter > triggerThreshold:
        
    #   get len counts
        lenCounts = df[skuColumnName].apply(len).value_counts()

    #   grab max count for len
        avLenMax = lenCounts.values.max()

    #   grab key for max count. This will be the actual len
        for key, value in lenCounts.items():
            if value == avLenMax:
                actualSkuLen = key

    #   add zeros to skus up to actualSkuLen. commentted out since this alters the df directly
#         lzdf[skuColumnName]= list(map(lambda x: x.zfill(actualSkuLen), lzdf[skuColumnName]))

    #   adds zeros to skus up to acutalSkuLen   
        for x in dataDict:
            sku = x[skuColumnName]
            sku = sku.zfill(actualSkuLen)
            x[skuColumnName] = sku
            
        lzDf = pd.DataFrame(dataDict)
        
    else:
        pass
    
    
    return lzDf

df1 = leading_zeros(data, "SKU")
df1

,Country,brandName,SKU,altSku,model,productName,upc,releaseDate,productPageUrl,imageURL,productGroup,category,foodType,brand,productType,packageSize,Unnamed: 16
0,US,KRAFT,00021000010615,NaN,US Kraft Heinz,Kraft 100% Grated Parmesan Cheese Shaker 24 oz...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--RlgA...,US Kraft Heinz,Cheese & Dairy,Cheese,KRAFT,PARMESAN CHEESE SHAKER,24.0,NaN
1,US,KRAFT,00021000014088,NaN,US Kraft Heinz,Kraft Asian Sesame Dressing 16 fl oz Bottle,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--6NCy...,US Kraft Heinz,"Sauces, Condiments & Salad Dressings",Salad Dressing,KRAFT,ASIAN SESAME DRESSING,16.0,NaN
2,US,KRAFT,00021000018567,NaN,US Kraft Heinz,Kraft Big Slice Mild Cheddar Natural Cheese Sl...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--Ish-...,US Kraft Heinz,Cheese & Dairy,Cheese,KRAFT,MILD CHEDDAR BIG SLICE,8.0,NaN
3,US,KRAFT,00021000018635,NaN,US Kraft Heinz,Kraft Big Slice Sharp Cheddar Natural Cheese S...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--pQ2S...,US Kraft Heinz,Cheese & Dairy,Cheese,KRAFT,SHARP CHEDDAR BIG SLICE,8.0,NaN
4,US,KRAFT,00021000028207,NaN,US Kraft Heinz,Kraft Honey Mustard Dressing 16 fl oz Bottle,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--z5lM...,US Kraft Heinz,"Sauces, Condiments & Salad Dressings",Salad Dressing,KRAFT,HONEY MUSTARD DRESSING,16.0,NaN
5,US,KRAFT,00021000031337,NaN,US Kraft Heinz,Kraft Slim Cut Sharp Cheddar 2% Milk Natural C...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--6ipz...,US Kraft Heinz,Cheese & Dairy,Cheese,KRAFT,SHARP CHEDDAR SLICES,7.0,NaN
6,US,VELVEETA,00021000039814,NaN,US Kraft Heinz,Velveeta Queso Blanco Flavor Cheese 16 slices ...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--mn83...,US Kraft Heinz,Cheese & Dairy,Cheese,VELVEETA,QUESO BLANCO SLICES,12.0,NaN
7,US,VELVEETA,00021000044429,NaN,US Kraft Heinz,Velveeta Original Cheese Sauce 3 - 4 oz Pouches,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--UZmS...,US Kraft Heinz,Cheese & Dairy,Cheese,VELVEETA,ORIGINAL CHEESE SAUCE,12.0,NaN
8,US,VELVEETA,00021000044436,NaN,US Kraft Heinz,Velveeta Queso Blanco Cheese Sauce 3 - 4 oz Po...,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--Q4J-...,US Kraft Heinz,Cheese & Dairy,Cheese,VELVEETA,QUESO BLANCO,12.0,NaN
9,US,VELVEETA,00021000044962,NaN,US Kraft Heinz,Velveeta Original Cheese Sauce 4 oz Box,NaN,NaN,NaN,http://images.salsify.com/image/upload/s--Z3da...,US Kraft Heinz,Cheese & Dairy,Cheese,VELVEETA,Original Cheese Sauce,4.0,NaN


In [52]:
# takes in df and column name. Coverts neg ints to positive. Returns corrected df.
def neg_num_check(df, columnName):
    
    dataDict = df.to_dict('records')
    
    for x in dataDict:
        try:
            num = int(x[columnName])
        except:
            pass
        if num < 0:
            x[columnName] = abs(num)
    
    correctedDf = pd.DataFrame(dataDict)
    
    return correctedDf


neg_num_check(data, "sku")

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
0,nl,Pamper,4015400833079,1 | 420blazit|wow,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833079,2019-07-20 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,33 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
1,NL,Pamperz,8001090268617,"1asdf, asdf|pipe",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090088185,2019-07-21 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,42 luiers (voor ongeveer 8 dagen),ES,NaN,NaN
2,GE,pampers,4015400833154,this is made of altskus,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336354,2019-07-22 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,46 luiers (voor ongeveer 9 dagen),ES,NaN,NaN
3,US,Pampers,4015400833154,"4015400833154, hat",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833154,2019-07-23 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 2 4/8 kg,58 luiers (voor ongeveer 11 dagen),ES,NaN,NaN
4,NL,Pampers,8001090268617,"1234, 232, 234",NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-12 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,100 luiers (voor ongeveer 20 dagen),ES,NaN,NaN
5,NL,Pampers,4015400833420,a|D|B,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833420,2019-07-25 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,112 luiers (voor ongeveer 23 dagen),ES,NaN,NaN
6,NL,Pampers,4015400833444,1|2|3,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833444,2019-07-26 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,136 luiers (voor ongeveer 28 dagen),ES,NaN,NaN
7,NL,Pampers,4015400566663,asdf,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400566663,2019-07-27 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,198 luiers (voor ongeveer 40 dagen),ES,NaN,NaN
8,NL,Pampers,4015400833260,hi wut-up,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833260,2019-07-28 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,30 luiers (voor ongeveer 6 dagen),ES,NaN,NaN
9,NL,Pampers,4015400833307,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,4015400833307,07/32/2019,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,34 luiers (voor ongeveer 7 dagen),ES,NaN,NaN


In [99]:
def selector_validation(df, selectorColumnNameList):
    
    for x in selectorList:
        a = outlier_detect(df,x)
#         print(a)
    
    return a

selectorList = ["size"]
b, c =  selector_validation(data,selectorList)
c

,Country,manufacturerName,sku,altSku,model,productName,upc,releaseDate,productPageUrl,imageUrl,productGroup,category,productCategory,productType,size,resourceLanguage,Column17,Column18
58,NL,Pampers,8001090298300,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090298300,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 7 15+ kg,17 luiers (voor ongeveer 4 dagen),ES,NaN,NaN
113,NL,Pampers,8001841072371,NaN,NL LUIERBROEKJES Pampers Baby-Dry Pants,LUIERBROEKJES Pampers Baby-Dry Pants,8001841072371,NaN,https://www.pampers.nl/producten/pampers-baby-...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry-pants,Maat 5 12/17 kg,144 luierbroekjes (voor ongeveer 36 dagen),ES,NaN,NaN
10,NL,Pampers,8001090336408,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336408,2019-07-08 00:00:00,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry,Maat 3 6/10 kg,38 luiers (voor ongeveer 8 dagen),ES,NaN,NaN
100,NL,Pampers,8001090706485,NaN,NL LUIERBROEKJES Pampers Baby-Dry Pants,LUIERBROEKJES Pampers Baby-Dry Pants,8001090706485,NaN,https://www.pampers.nl/producten/pampers-baby-...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry-pants,Maat 3 6/11 kg,80 luierbroekjes (voor ongeveer 18 dagen),ES,NaN,NaN
126,NL,Pampers,4015400622253,NaN,NL DOEKJES Pampers Sensitive Babydoekjes,DOEKJES Pampers Sensitive Babydoekjes,4015400622253,NaN,https://www.pampers.nl/producten/pampers-sensi...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Doekjes,pampers-sensitive-babydoekjes,NaN,504 doekjes,ES,NaN,NaN
168,NL,Pampers,8001090359629,NaN,NL LUIERBROEKJES Pampers Premium Protection Pants,LUIERBROEKJES Pampers Premium Protection Pants,8001090359629,NaN,https://www.pampers.nl/producten/pampers-premi...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-premium-protection-pants,Maat 6 15+ kg,48 luierbroekjes (voor ongeveer 12 dagen),ES,NaN,NaN
76,NL,Pampers,4015400849193,NaN,NL LUIERS Pampers Premium Protection,LUIERS Pampers Premium Protection,4015400849193,NaN,https://www.pampers.nl/producten/pampers-premi...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-premium-protection,Maat 3 6/10 kg,66 luiers (voor ongeveer 13 dagen),ES,NaN,NaN
99,NL,Pampers,4015400749608,NaN,NL LUIERBROEKJES Pampers Baby-Dry Pants,LUIERBROEKJES Pampers Baby-Dry Pants,4015400749608,NaN,https://www.pampers.nl/producten/pampers-baby-...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby-dry-pants,Maat 3 6/11 kg,44 luierbroekjes (voor ongeveer 10 dagen),ES,NaN,NaN
19,NL,Pampers,8001090336446,NaN,NL LUIERS Pampers Baby-Dry,LUIERS Pampers Baby-Dry,8001090336446,NaN,https://www.pampers.nl/producten/pampers-baby-dry,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Luiers,pampers-baby dry,Maat 4 9/14 kg,34 luiers (voor ongeveer 8 dagen),ES,NaN,NaN
157,NL,Pampers,4015400622840,NaN,NL DOEKJES Pampers Fresh Clean Babydoekjes,DOEKJES Pampers Fresh Clean Babydoekjes,4015400622840,NaN,https://www.pampers.nl/producten/pampers-fresh...,http://azcdn.multibrand.pgsitecore.com/nl-NL/-...,NL Pampers,Doekjes,pampers-fresh-clean-babydoekjes,NaN,64 doekjes,ES,NaN,NaN


In [37]:
# simiplier solution found. Not using.
# Returns True for store feeds false for product feeds
def isStoreFeed(df):
    
    isStoreFeed = False
    
    columnNames = df.columns.tolist()
    
    r = re.compile(".*store[^'+]|.*seller[^'+]")
    
    matchedList = list(filter(r.match, columnNames))
    
    if len(matchedList) >= 3:
        isStoreFeed = True
    
    return isStoreFeed

isStoreFeed(sfData)

True

In [115]:
# removes lat and long outside the actual bounds
def latLongValidation(df,latColumn,longColumn):
    
    lat = df[latColumn]
    long = df[longColumn]
    
    lat = map(float, lat)
    long = map(float, long)
    
    badLat = []
    badLong = []
    
    [badLat.append(ind) for ind,x in enumerate(lat) if x >= 90 or x <= -90 ]
    [badLong.append(ind) for ind,x in enumerate(long) if x >= 180 or x <= -180 ]
    
    badLatLong = list(set(badLat + badLong))
    
    cleanDf = df.drop(badLatLong)
    badDf = df.iloc[badLatLong]
    
    return cleanDf,badDf

latLongValidation(sfData,"latitude","longitude")

(     Country         sellerName sellerID                     storeName  \
 0         CA  Acklands Grainger   100000  Acklands Grainger Inc - AC0    
 1         CA  Acklands Grainger   100000  Acklands Grainger Inc - AA9    
 2         CA  Acklands Grainger   100000  Acklands Grainger Inc - AC4    
 3         CA  Acklands Grainger   100000  Acklands Grainger Inc - AC1    
 4         CA  Acklands Grainger   100000  Acklands Grainger Inc - AC2    
 ...      ...                ...      ...                           ...   
 8657      US          White Cap   110600      White Cap - Seattle - 46   
 8658      US          White Cap   110600      White Cap - Spokane - 47   
 8659      US          White Cap   110600      White Cap - Tacoma - 506   
 8660      US          White Cap   110600    White Cap - Cheyenne - 501   
 8661      US      Wylaco Supply   110700            Wylaco Supply - 00   
 
              storeID                            productGroup  sku  \
 0               AC0        